In [1]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

import time
import random

from bs4 import BeautifulSoup
import requests as req
from fake_useragent import UserAgent

import json
import w3lib.html

from lxml import html
from tqdm import tqdm

In [2]:
class Scraper():
    def __init__(self):
        self.ua = UserAgent()
        self.headers = req.utils.default_headers()
    
    def index_archive(self):
        # Create a list of all monthly sitemap urls
        date_input_format = '%m-%Y'
        date_nrc_format = '%Y-%m'

        # start_date = '10-1970'
        # test start date
        start_date = '2-2021'
        end_date = datetime.today().strftime(date_input_format)

        def diff_month(d1, d2):
            return (d1.year - d2.year) * 12 + d1.month - d2.month

        start = datetime.strptime(start_date, date_input_format)
        end = datetime.strptime(end_date, date_input_format)

        date_range= [start + relativedelta(months=x) for x in range(0, diff_month(end, start)+1)]

        self.archive_index = [f"https://www.nrc.nl/sitemap/{date.strftime(date_nrc_format)}.xml" for date in date_range]
        
    def index_urls(self):
        
        def fetchArchiveURL(url):
            document = {}

            try:
                # Random User Agent
                self.headers.update({'User-Agent': self.ua.random,})
                r = req.get(url)
                r.raise_for_status()
            except req.exceptions.HTTPError as err:
                pass
            else:        
                document['url'] = url
                document['raw_xml'] = r.content
                document['in_cache_date'] = datetime.utcnow()

            return document
        
        def fetchArticleURL(url):
            doc = fetchArchiveURL(url)
            raw_xml = doc['raw_xml']
            soup = BeautifulSoup(raw_xml, 'xml')

            article_urls = []
            locs = soup.find_all('loc')
            for loc in locs:
                article_urls.append(loc.get_text())

            return article_urls
        
        self.article_urls = []
        print('Indexing urls: ')
        for url in tqdm(self.archive_index):
            # Simulate human usage 
            time.sleep(random.randint(0, 3))
            self.article_urls.extend(fetchArticleURL(url))
    
    def crawl_articles(self):
        
        def fetchArticle(url):
            document = {}

            try:
                # Random User Agent
                self.headers.update({'User-Agent': self.ua.random,})
                
                r = req.get(url)
                r.raise_for_status()

            except req.exceptions.HTTPError as err:
                pass

            else:        
                document['url'] = url
                document['raw_html'] = r.content
                document['in_cache_date'] = datetime.utcnow()
                document['parsed'] = 0

            return document
        
        self.articles = []
        print('Crawling articles: ')
        for url in tqdm(self.article_urls):
            # Simulate human usage
            time.sleep(random.randint(0, 3))
            self.articles.append(fetchArticle(url))
    
    def clean_articles(self):
        
        def parse_article(r):

            a = {}

            a['url']  = r['url']

            tree = html.fromstring(r['raw_html'])

            try:
                a['timestamp'] = tree.xpath('/html/body/main/div[2]/div/article/div/div/div[1]/div/section/div[1]/div//time[@class="article__byline__text"]/@datetime')[0].strip()
            except: 
                a['timestamp'] = None
            try:
                a['title'] = tree.xpath('//h1/text()')[0].strip()
            except:
                a['title'] = None
            try:
                a['publisherID'] = tree.xpath('/html/body/main/div[2]/div/article/div/div/div[1]/div/section/div[1]/div/div[1]/ul/li/a/text()')[0].strip()
            except:
                a['publisherID'] = None
            try:
                a['cleantext'] = ''.join(tree.xpath('//div[@class="content article__content"]//p/text()')[:-3])
            except:
                a['cleantext'] = None
            try:
                a['category'] = tree.xpath('/html/body/a/div/div/h2/text()')[0].strip()
            except:
                a['category'] = None

            return a
        
        print('Cleaning articles:')
        self.cleaned_articles = []
        for article in tqdm(self.articles):
            self.cleaned_articles.append(parse_article(article))
    
    def save_json(self):
        
        with open('NRC.json', 'w') as file:
            json.dump(self.cleaned_articles, file, indent=4)
            
    

In [3]:
scraper = Scraper()


{'User-Agent': 'python-requests/2.25.1', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}


In [4]:
scraper.index_archive()

In [5]:
scraper.index_urls()

  0%|                                                                                           | 0/14 [00:00<?, ?it/s]

Indexing urls: 


100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:30<00:00,  2.15s/it]


In [6]:
scraper.crawl_articles()

  0%|                                                                                        | 0/38839 [00:00<?, ?it/s]

Crawling articles: 


 14%|█████████▊                                                              | 5311/38839 [2:50:29<17:56:16,  1.93s/it]


TooManyRedirects: Exceeded 30 redirects.

In [7]:
scraper.clean_articles()

  1%|▉                                                                              | 66/5311 [00:00<00:15, 329.73it/s]

Cleaning articles:


100%|█████████████████████████████████████████████████████████████████████████████| 5311/5311 [00:15<00:00, 352.45it/s]


In [8]:
scraper.save_json()